In [1]:
import pandas as pd
from scipy.stats import ttest_ind
import re
from statsmodels.stats.multitest import multipletests
import numpy as np
import matplotlib as plt
import pandas as pd

In [2]:
def univariate_analysis(row):
    group_data = {}
    
    # Extract group data based on the pattern
    for col in row.index:
        match = re.match(r"batch\d+_(C|S|QC)\d+_rep\d+", col)
        if match:
            # Extract the group part (C, S, or QC)
            group_part = match.group(1)
            # Initialize list if key does not exist
            if group_part not in group_data:
                group_data[group_part] = []
            # Append the value to the list for the group
            group_data[group_part].append(row[col])

    # Check if all groups have data (avoid errors with missing values)
    if len(group_data) < 3:
        return {"all_groups_not_present": True}

    # Perform t-tests between each pair of groups
    comparisons = []
    p_values = []
    fold_changes = [] 
    FC_comparisons = []
    groups = list(group_data.keys())
    
    for i in range(len(groups)):
        for j in range(i + 1, len(groups)):
            group1, group2 = groups[i], groups[j]
            data1, data2 = group_data[group1], group_data[group2]
            
            # Ensure data is numeric and convert to appropriate format
            data1 = [float(val) for val in data1 if is_numeric(val)]
            data2 = [float(val) for val in data2 if is_numeric(val)]
            
            if len(data1) > 1 and len(data2) > 1:  # Ensure sufficient data points
                statistic, p_value = ttest_ind(data1, data2)
                p_values.append(p_value)
                comparisons.append(f"{group1} vs {group2}")
            else:
                p_values.append(np.nan)
                FC_comparisons.append(f"{group1} vs {group2}")

            if len(data1) > 0 and len(data2) > 0:
                fold_change = np.mean(data2) / np.mean(data1)  # Calculate the fold change
                log2_fold_change = np.log2(fold_change)  # Calculate the log2 fold change
                fold_changes.append(log2_fold_change)
                FC_comparisons.append(f"{group1} vs {group2} Fold Change")
            else:
                log2_fold_change = np.nan
                fold_changes.append(log2_fold_change)
                FC_comparisons.append(f"{group1} vs {group2} Fold Change")



    # Convert p-values to a list for multiple testing correction
    p_value_list = [p for p in p_values if not np.isnan(p)]
    
    # Apply Benjamini-Hochberg correction
    bh_corrected_p_values = multipletests(p_value_list, method='fdr_bh')[1]
    
    # Apply Bonferroni correction
    bonferroni_corrected_p_values = multipletests(p_value_list, method='bonferroni')[1]

    # Update p_values list with corrected p-values
    bh_corrected_p_dict = {}
    bonferroni_corrected_p_dict = {}
    idx = 0
    for i, p in enumerate(p_values):
        if not np.isnan(p):
            bh_corrected_p_dict[comparisons[i]] = bh_corrected_p_values[idx]
            bonferroni_corrected_p_dict[comparisons[i]] = bonferroni_corrected_p_values[idx]
            idx += 1
        else:
            bh_corrected_p_dict[comparisons[i]] = "Insufficient data"
            bonferroni_corrected_p_dict[comparisons[i]] = "Insufficient data"
    
    # Create DataFrames to store p-values with the row index
    df_p_values = pd.DataFrame(data=[p_values], index=[row.name], columns=comparisons)
    df_bh_corrected_p_values = pd.DataFrame(data=[bh_corrected_p_dict.values()], index=[row.name], columns=comparisons)
    df_bonferroni_corrected_p_values = pd.DataFrame(data=[bonferroni_corrected_p_dict.values()], index=[row.name], columns=comparisons)
    df_Fold_changes = pd.DataFrame(data=[fold_changes], index=[row.name],columns=FC_comparisons)

    # Calculate descriptive statistics
    descriptive_stats = {group: {"mean": np.mean(data), "std": np.std(data)} for group, data in group_data.items() if len(data) > 1}
    
    return {
        "raw_p_values": df_p_values,
        "bh_corrected_p_values": df_bh_corrected_p_values,
        "bonferroni_corrected_p_values": df_bonferroni_corrected_p_values,
        "descriptive_statistics": descriptive_stats,
        "fold changes": df_Fold_changes
    }

def is_numeric(val):
    try:
        float(val)
        return True
    except ValueError:
        return False

In [3]:
MALDIquant_df = pd.read_csv(r"MALDIquant_Tol5e-6_keep400_C_Filtered_BC_Clustered_Feature_Matrix.csv",index_col=0)

MALDIquant_raw_p_values_df = pd.DataFrame()
MALDIquant_benjamini_corrected_p_values_df = pd.DataFrame()
MALDIquant_bonferroni_corrected_p_values_df = pd.DataFrame()
MALDIquant_FC = pd.DataFrame()

for index, row in MALDIquant_df.iterrows():
    result = univariate_analysis(row)
    if "all_groups_not_present" in result:
        continue
    
    MALDIquant_raw_p_values_df = pd.concat([MALDIquant_raw_p_values_df, result["raw_p_values"]])
    MALDIquant_benjamini_corrected_p_values_df = pd.concat([MALDIquant_benjamini_corrected_p_values_df, result["bh_corrected_p_values"]])
    MALDIquant_bonferroni_corrected_p_values_df = pd.concat([MALDIquant_bonferroni_corrected_p_values_df, result["bonferroni_corrected_p_values"]])
    MALDIquant_FC = pd.concat([MALDIquant_FC,result["fold changes"]])



MALDIquant_raw_p_values_df.to_csv("MALDIquant_p_values.csv")
MALDIquant_benjamini_corrected_p_values_df.to_csv("MALDIquant_benjamini_p_values.csv")
MALDIquant_bonferroni_corrected_p_values_df.to_csv("MALDIquant_bonferroni_p_values.csv")
MALDIquant_FC.to_csv("MALDIquant_FC.csv")

MALDIquant_benjamini_corrected_p_values_df.head()

,C vs QC,C vs S,QC vs S
96.998157,6.343946e-01,6.343946e-01,6.343946e-01
98.992727,8.933700e-11,2.166624e-16,1.507108e-01
563.551042,3.536488e-11,3.680785e-01,2.849096e-30
103.037034,3.845935e-01,6.428598e-01,3.921197e-01
104.107932,2.168794e-01,1.762702e-05,3.353113e-03


In [4]:
MALDIquant_df.head()

,batch01_C03_rep01,batch01_C03_rep02,batch01_C03_rep03,batch01_C04_rep01,batch01_C04_rep02,batch01_C04_rep03,batch01_C05_rep01,batch01_C05_rep02,batch01_C05_rep03,batch01_C06_rep01,...,batch08_S07_rep03,batch08_S08_rep01,batch08_S08_rep02,batch08_S08_rep03,batch08_S09_rep01,batch08_S09_rep02,batch08_S09_rep03,batch08_S10_rep01,batch08_S10_rep02,batch08_S10_rep03
feature_id,,,,,,,,,,,,,,,,,,,,,
96.998157,5480.463424,6284.636072,6559.242925,3819.696916,9481.530799,4680.194857,4310.523854,5550.732676,7157.773010,2837.881153,...,24818.504802,11791.651294,24423.388536,14730.457738,51824.956991,15385.642868,25440.242777,7080.243753,8135.958035,6526.082375
98.992727,3757.180703,6513.143026,3587.653786,3428.794498,3534.499940,3930.772753,3448.829184,3730.545269,4805.162338,3474.928037,...,14152.079115,15071.714767,9940.736140,22167.954994,17130.839486,9374.754582,27493.145544,20464.076206,11937.757483,13628.220092
563.551042,118647.178661,119882.455729,90938.019138,71326.758422,87178.230270,126481.904948,68124.205899,78375.218127,172334.715445,83278.131992,...,177375.243422,148303.209189,135254.555914,169769.224644,266929.638221,300026.531250,306079.567308,186939.562790,147343.649933,401323.203347
103.037034,3508.317870,11462.164115,5639.467968,6909.680284,9413.737296,8105.561920,4225.042278,5692.263387,9075.770606,4810.820085,...,38122.605076,11082.566633,5671.553479,12833.720547,36142.503517,22656.445463,41062.503389,10002.173382,7650.373374,14021.611701
104.107932,5786.093613,9839.905944,5065.227211,5543.583850,7566.350495,6611.441810,4417.083728,4354.950531,13690.992875,3931.766549,...,9128.629973,6780.236127,5693.875622,6293.213784,12289.032205,6297.173121,6232.884764,5180.377064,6562.365882,5776.035787


In [5]:
DIMSpy_df = pd.read_csv(r"DIMSpy_SNR3.5_ppm555_NC_BC_Clustered_Feature_Matrix.csv",index_col=0)

DIMSpy_raw_p_values_df = pd.DataFrame()
DIMSpy_benjamini_corrected_p_values_df = pd.DataFrame()
DIMSpy_bonferroni_corrected_p_values_df = pd.DataFrame()
DIMSpy_FC = pd.DataFrame()

for index, row in DIMSpy_df.iterrows():
    result = univariate_analysis(row)
    
    if "all_groups_not_present" in result:
        continue
    
    DIMSpy_raw_p_values_df = pd.concat([DIMSpy_raw_p_values_df, result["raw_p_values"]])
    DIMSpy_benjamini_corrected_p_values_df = pd.concat([DIMSpy_benjamini_corrected_p_values_df, result["bh_corrected_p_values"]])
    DIMSpy_bonferroni_corrected_p_values_df = pd.concat([DIMSpy_bonferroni_corrected_p_values_df, result["bonferroni_corrected_p_values"]])
    DIMSpy_FC = pd.concat([DIMSpy_FC,result["fold changes"]])


DIMSpy_raw_p_values_df.to_csv("DIMSpy_p_values.csv")
DIMSpy_benjamini_corrected_p_values_df.to_csv("DIMSpy_benjamini_p_values.csv")
DIMSpy_bonferroni_corrected_p_values_df.to_csv("DIMSpy_bonferroni_p_values.csv")
DIMSpy_FC.to_csv("DIMSpy_FC.csv")

In [6]:
DIMSpy_df.head()

,batch01_C03_rep01,batch01_C04_rep01,batch01_C05_rep01,batch01_C06_rep01,batch01_C07_rep01,batch01_C08_rep01,batch01_C09_rep01,batch01_C10_rep01,batch01_QC01_rep01,batch01_QC02_rep01,...,batch08_S01_rep01,batch08_S02_rep01,batch08_S03_rep01,batch08_S04_rep01,batch08_S05_rep01,batch08_S06_rep01,batch08_S07_rep01,batch08_S08_rep01,batch08_S09_rep01,batch08_S10_rep01
feature_id,,,,,,,,,,,,,,,,,,,,,
70.033971,10.631759,10.614377,10.474018,10.658824,10.782228,10.446394,10.459039,10.744663,10.412898,10.413402,...,9.747327,9.927700,9.358759,9.496502,9.373675,9.301410,9.641347,9.194206,8.700068,9.150718
132.076962,17.268105,17.331174,17.418973,17.278835,17.164006,17.050572,17.059195,17.080020,16.942215,16.942624,...,16.637979,16.863048,16.555035,16.719140,16.399103,16.190320,16.344014,16.361188,16.164959,16.380558
74.001836,9.417421,9.281649,9.471803,9.481754,9.321361,9.557678,9.735218,9.692461,10.141318,10.151134,...,10.276304,9.779407,9.825303,9.648669,9.954353,10.089386,9.637076,9.816623,10.139225,9.969993
90.054935,9.727045,10.540320,9.671431,9.803895,10.048381,9.888251,9.851125,10.011613,9.903772,9.869963,...,9.568852,9.437586,8.397088,7.896544,8.399869,6.933846,5.652377,5.204431,5.078440,4.950542
97.004965,4.756816,4.643302,5.913833,5.246071,6.695382,7.579036,6.891744,9.565967,10.655826,10.656309,...,11.521065,10.791103,10.224382,10.379528,11.276647,10.828267,9.889373,10.114725,10.151510,10.065625


In [7]:
MZmine_df = pd.read_csv(r"MZmine_MinF7.5E3_NT3.5_Tol555_NC_BC_Clustered_Feature_Matrix.csv", index_col=0)

MZmine_raw_p_values_df = pd.DataFrame()
MZmine_benjamini_corrected_p_values_df = pd.DataFrame()
MZmine_bonferroni_corrected_p_values_df = pd.DataFrame()
MZmine_FC = pd.DataFrame()

for index, row in MZmine_df.iterrows():
    result = univariate_analysis(row)
    
    if "all_groups_not_present" in result:
        continue
    
    MZmine_raw_p_values_df = pd.concat([MZmine_raw_p_values_df, result["raw_p_values"]])
    MZmine_benjamini_corrected_p_values_df = pd.concat([MZmine_benjamini_corrected_p_values_df, result["bh_corrected_p_values"]])
    MZmine_bonferroni_corrected_p_values_df = pd.concat([MZmine_bonferroni_corrected_p_values_df, result["bonferroni_corrected_p_values"]])
    MZmine_FC = pd.concat([MZmine_FC,result["fold changes"]])

MZmine_raw_p_values_df = MZmine_raw_p_values_df.rename(columns={'QC vs C': 'C vs QC'})
MZmine_benjamini_corrected_p_values_df = MZmine_benjamini_corrected_p_values_df.rename(columns={'QC vs C': 'C vs QC'})
MZmine_bonferroni_corrected_p_values_df = MZmine_bonferroni_corrected_p_values_df.rename(columns={'QC vs C': 'C vs QC'})


MZmine_raw_p_values_df.to_csv("MZmine_p_values.csv")
MZmine_benjamini_corrected_p_values_df.to_csv("MZmine_benjamini_p_values.csv")
MZmine_bonferroni_corrected_p_values_df.to_csv("MZmine_bonferroni_p_values.csv")
MZmine_FC.to_csv("MZmine_FC.csv")

c:\Users\adamg\OneDrive - The University of Liverpool\Project Work\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


In [8]:
MZmine_df.head()

,batch01_QC01_rep03,batch01_QC01_rep01,batch01_QC01_rep02,batch01_QC02_rep01,batch05_QC20_rep03,batch05_QC20_rep01,batch06_QC25_rep01,batch05_QC20_rep02,batch07_QC30_rep02,batch06_QC25_rep02,...,batch04_S10_rep03,batch04_QC19_rep03,batch04_S08_rep02,batch04_QC19_rep02,batch04_QC19_rep01,batch04_S08_rep03,batch04_C04_rep02,batch04_S09_rep03,batch04_C04_rep03,batch04_S10_rep02
feature_id,,,,,,,,,,,,,,,,,,,,,
101.79019,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,...,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889
101.75289,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,...,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889
143.01041,17.675890,17.675890,17.675890,17.675890,17.675890,17.675890,17.675890,17.675890,17.675890,17.675890,...,17.675890,17.675890,17.675890,17.675890,17.675890,17.675890,17.675890,17.675890,17.675890,17.675890
101.76158,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,...,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889
101.78959,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,...,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889,17.675889


In [9]:
MZmine_raw_p_values_df.columns

Index(['C vs QC', 'QC vs S', 'C vs S'], dtype='object')

In [10]:
dataframes = [
    MALDIquant_raw_p_values_df,MALDIquant_bonferroni_corrected_p_values_df,MALDIquant_benjamini_corrected_p_values_df,
    DIMSpy_raw_p_values_df,DIMSpy_bonferroni_corrected_p_values_df,DIMSpy_benjamini_corrected_p_values_df,
    MZmine_raw_p_values_df,MZmine_bonferroni_corrected_p_values_df,MZmine_benjamini_corrected_p_values_df
]

dataframes_labels = [
    "MALDIquant_raw_p_values_df", "MALDIquant_bonferroni_corrected_p_values_df", "MALDIquant_benjamini_corrected_p_values_df",
    "DIMSpy_raw_p_values_df", "DIMSpy_bonferroni_corrected_p_values_df", "DIMSpy_benjamini_corrected_p_values_df",
    "MZmine_raw_p_values_df", "MZmine_bonferroni_corrected_p_values_df", "MZmine_benjamini_corrected_p_values_df"]

In [11]:

def process_dataframe(df, label):
    columns = ["C vs QC", "C vs S", "QC vs S"]
    print(label)
    for column in columns:
        df[column] = pd.to_numeric(df[column], errors='coerce')

        std_dev = df[column].std()
        mean = df[column].mean()
        # Count of values below 0.05 for the specific column
        count_below_0_05 = (df[column] < 0.05).sum()
        
        print(f"Column: {column}")
        print(f"  Standard Deviation: {std_dev:.4f}")
        print(f"  Mean: {mean:.4f}")
        print(f"  Count of values below 0.05: {count_below_0_05}")
        print(f"  Total Values: {len(df)}")
        print()

for i in range(len(dataframes)):
  process_dataframe(dataframes[i],dataframes_labels[i])

MALDIquant_raw_p_values_df
Column: C vs QC
  Standard Deviation: 0.2630
  Mean: 0.1686
  Count of values below 0.05: 155
  Total Values: 260

Column: C vs S
  Standard Deviation: 0.2569
  Mean: 0.1536
  Count of values below 0.05: 162
  Total Values: 260

Column: QC vs S
  Standard Deviation: 0.2684
  Mean: 0.1470
  Count of values below 0.05: 177
  Total Values: 260

MALDIquant_bonferroni_corrected_p_values_df
Column: C vs QC
  Standard Deviation: 0.4207
  Mean: 0.3240
  Count of values below 0.05: 139
  Total Values: 260

Column: C vs S
  Standard Deviation: 0.4034
  Mean: 0.2916
  Count of values below 0.05: 145
  Total Values: 260

Column: QC vs S
  Standard Deviation: 0.3982
  Mean: 0.2588
  Count of values below 0.05: 164
  Total Values: 260

MALDIquant_benjamini_corrected_p_values_df
Column: C vs QC
  Standard Deviation: 0.2846
  Mean: 0.1940
  Count of values below 0.05: 148
  Total Values: 260

Column: C vs S
  Standard Deviation: 0.2825
  Mean: 0.1778
  Count of values below 